**How do those models work?**

Easy! We will cover it on the next course. Now, we focus on workflow of modeling.

**Note on Surprise Library**

To model the personalized RecSys, we will use a well-defined library called with `surprise`. See the [Surprise Docs.](https://surprise.readthedocs.io/en/stable/index.html)

In [ ]:
# Install the library
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Then load the library

In [ ]:
import surprise

Now let's start modeling.

### Load the Data
---

Why we do this **again**? Because we works on a specific library that need specific input format.

So, let's do this

In [ ]:
# Import some library
from surprise import Dataset, Reader

Initiate the rating scale

In [ ]:
reader = Reader(rating_scale = (1, 5))
reader

Initiate the data. It must be on format `userId`, `itemId`, and `ratings`, respectively.

In [ ]:
utility_data = Dataset.load_from_df(
                    df = rating_data[['userId', 'movieId', 'rating']].copy(),
                    reader = reader
                )

utility_data

Let's print the data

In [ ]:
utility_data.df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
rating_data.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


Nice! we have similar dataset

### Split Train-Test
---

We then split the train-test data. We use the similar logic from previous section

In [ ]:
# Load library for deep copy
import copy

In [ ]:
# Create a function
def train_test_split(utility_data, test_size, random_state):
    """
    Train test split the data
    ref: https://surprise.readthedocs.io/en/stable/FAQ.html#split-data-for-unbiased-estimation-py

    Parameters
    ----------
    utility_data : Surprise utility data
        The sample of whole data set

    test_size : float, default=0.2
        The test size

    random_state : int, default=42
        For reproducibility

    Returns
    -------
    full_data : Surprise utility data
        The new utility data

    train_data : Surprise format
        The train data

    test_data : Surprise format
        The test data
    """
    # Deep copy the utility_data
    full_data = copy.deepcopy(utility_data)

    # Generate random seed
    np.random.seed(random_state)

    # Shuffle the raw_ratings for reproducibility
    raw_ratings = full_data.raw_ratings
    np.random.shuffle(raw_ratings)

    # Define the threshold
    threshold = int((1-test_size) * len(raw_ratings))

    # Split the data
    train_raw_ratings = raw_ratings[:threshold]
    test_raw_ratings = raw_ratings[threshold:]

    # Get the data
    full_data.raw_ratings = train_raw_ratings
    train_data = full_data.build_full_trainset()
    test_data = full_data.construct_testset(test_raw_ratings)

    return full_data, train_data, test_data


In [ ]:
# Split the data
full_data, train_data, test_data = train_test_split(utility_data,
                                                    test_size = 0.2,
                                                    random_state = 42)

In [ ]:
# Validate the splitting
train_data.n_ratings, len(test_data)

(80668, 20168)

Great! The test size is around 20% of all dataset

Now we are ready to create the model.

### Create the Model
---

We want to create two models & get the best model.

To do that, we need to perform a cross validation thus we evaluate the model objectively.

In [ ]:
# Load the model library
# i.e. Baseline, KNN, and SVD
from surprise import AlgoBase, KNNBasic, SVD

Then, create the model object

Our Baseline Model just simply predict ratings using mean from all training data provided to model .

Since `surprise` library does not provide mean prediction model , we have to create custom algorithm first.

Guide `https://surprise.readthedocs.io/en/stable/building_custom_algo.html`

In [ ]:
class MeanPrediction(AlgoBase):
    '''Baseline prediction. Return global mean as prediction'''
    def __init__(self):
        AlgoBase.__init__(self)

    def fit(self, trainset):
        '''Fit the train data'''
        AlgoBase.fit(self, trainset)

    def estimate(self, u, i):
        '''Perform the estimation/prediction.'''
        est = self.trainset.global_mean
        return est

In [ ]:
# Create baseline model
model_baseline = MeanPrediction()
model_baseline

In [ ]:
# Create Neighbor-based model -- K-Nearest Neighbor
model_knn = KNNBasic(random_state=42)
model_knn

In [ ]:
# Create matrix factorization model -- SVD-like
model_svd = SVD(n_factors=100, random_state=42)
model_svd

Next, fit & cross-validate the model.

### Find the Best Model with Cross-Validation
---

In [ ]:
# Import the cross validation module
from surprise.model_selection import cross_validate

To perform cross validate
`cross_validate(algo, data, cv, measures)`

1. `algo` = Surprise model
2. `data` = Surprise format data
3. `cv` = number of fold of cross validation
4. `measures` = metric to measure model performance



First, we will do a $5$-fold cross validation. Next, we measure the performance using RMSE metric because we want to predict the rating (continuous data type)

**Baseline** model

In [ ]:
# Use full_data for cross validation
# Your results could be different because
# there is no random seed stated within this functions
cv_baseline = cross_validate(algo = model_baseline,
                             data = full_data,
                             cv = 5,
                             measures = ['rmse'])

In [ ]:
# Extract CV results
cv_baseline_rmse = cv_baseline['test_rmse'].mean()
cv_baseline_rmse

1.0420126162986096

**K-Nearest Neighbor** model

In [ ]:
cv_knn = cross_validate(algo = model_knn,
                        data = full_data,
                        cv = 5,
                        measures = ['rmse'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
# Extract CV results
cv_knn_rmse = cv_knn['test_rmse'].mean()
cv_knn_rmse

0.9589057865687737

**SVD-like** model

In [ ]:
cv_svd = cross_validate(algo = model_svd,
                        data = full_data,
                        cv = 5,
                        measures = ['rmse'])

In [ ]:
# Extract CV results
cv_svd_rmse = cv_svd['test_rmse'].mean()
cv_svd_rmse

0.8793607027046333

**Performance comparison**

We can summarize the performance

In [ ]:
summary_df = pd.DataFrame({'Model': ['Baseline', 'KNN', 'SVD'],
                           'CV Performance - RMSE': [cv_baseline_rmse, cv_knn_rmse, cv_svd_rmse]})

summary_df

,Model,CV Performance - RMSE
0,Baseline,1.042013
1,KNN,0.958906
2,SVD,0.879361


The best model is **SVD** with better CV performance, i.e. lower RMSE

Finally, we retrain the SVD model with the whole train dataset

In [ ]:
# Create object
model_best = SVD(n_factors=100, random_state=42)

# Retrain on whole train dataset
model_best.fit(train_data)

### Evaluate the Best Model
---

After finding the best model, we can sanity check the performance on the test dataset

In [ ]:
# import performance library
from surprise import accuracy

Next, we predict the test set using our best model

In [ ]:
test_pred = model_best.test(test_data)
test_rmse = accuracy.rmse(test_pred)
test_rmse

RMSE: 0.8807


0.8807462819979623

To summarize

In [ ]:
summary_test_df = pd.DataFrame({'Model' : ['SVD'],
                                'RMSE-CV': [cv_svd_rmse],
                                'RMSE-Test': [test_rmse]})

summary_test_df

,Model,RMSE-CV,RMSE-Test
0,SVD,0.879361,0.880746


Great predictions!

# **4. Personalized Recommender Systems**: Predictions
---

Decision Process is to recommend items to user
from our trained model

<image src="https://www.mdpi.com/applsci/applsci-10-05510/article_deploy/html/images/applsci-10-05510-g001.png" image>




1. SVD Like-Recommender System
The Model will lookup latent factor (vector) both movie and user factor based on its' userId and movieId. Rating prediction would be multiplication between user factor and item factor. After that the predicted rating will be ordered and will be picked Top K Highest Predicted rating and recommend the movie to user.

Also if user already seen a movie it could be removed.


## Predict with Best Model : SVD-Like Recommender System

In [ ]:
# Recommendation based on SVD-like Recommender System Model
# We will try to recommend on sample userid ,userId 1 & 99

# We can use model_best.predict method
help(model_best.predict)

Help on method predict in module surprise.prediction_algorithms.algo_base:

predict(uid, iid, r_ui=None, clip=True, verbose=False) method of surprise.prediction_algorithms.matrix_factorization.SVD instance
    Compute the rating prediction for given user and item.
    
    The ``predict`` method converts raw ids to inner ids and then calls the
    ``estimate`` method which is defined in every derived class. If the
    prediction is impossible (e.g. because the user and/or the item is
    unknown), the prediction is set according to
    :meth:`default_prediction()
    <surprise.prediction_algorithms.algo_base.AlgoBase.default_prediction>`.
    
    Args:
        uid: (Raw) id of the user. See :ref:`this note<raw_inner_note>`.
        iid: (Raw) id of the item. See :ref:`this note<raw_inner_note>`.
        r_ui(float): The true rating :math:`r_{ui}`. Optional, default is
            ``None``.
        clip(bool): Whether to clip the estimation into the rating scale.
            For exampl

`model_best.predict` has argument
- `uid` (i.e., the `userId`) and
- `iid` (i.e., the item ID or `movieId`)

### Let's predict what is the rating of user 9 to movie 10

In [ ]:
sample_prediction = model_svd.predict(uid = 9,
                                      iid = 10)

In [ ]:
sample_prediction

Prediction(uid=9, iid=10, r_ui=None, est=3.2224705698626694, details={'was_impossible': False})

The results tell us
- `r_ui` : actual rating --> `None`, means user 9 have yet rated movie 10
- `est` : the estimated rating from our model
- `details` : whether prediction is impossible or not. So it's possible to predict.

### Let's predict all the unseen/unrated movie by userId 9

**First, we find the unrated movie id from user id 9**

In [ ]:
# Get unique movieId
unique_movie_id = set(rating_data['movieId'])
print(unique_movie_id)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 131098, 31, 32, 131104, 34, 30, 36, 32799, 38, 39, 40, 41, 65577, 43, 44, 45, 46, 47, 48, 65585, 50, 49, 52, 53, 54, 55, 65588, 57, 58, 98361, 60, 61, 62, 65596, 64, 65, 66, 131130, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 163925, 92, 93, 94, 95, 65631, 97, 96, 99, 100, 101, 102, 103, 104, 105, 106, 107, 32875, 65642, 110, 111, 112, 113, 108, 65651, 116, 117, 118, 119, 121, 122, 123, 32892, 125, 126, 128, 129, 32898, 132, 135, 137, 32906, 140, 141, 163981, 144, 145, 146, 147, 65682, 148, 150, 151, 163985, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 131237, 166, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 187, 98491, 189, 190, 191, 188, 193, 194, 195, 196, 98499, 198, 199, 98503, 201, 65738, 203, 204, 205, 65740, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 

In [ ]:
# Get movieId that is rated by user id 9
rated_movie_id = set(rating_data.loc[rating_data['userId']==9, 'movieId'])
print(rated_movie_id)

{3328, 4993, 5378, 5507, 5890, 5891, 5893, 1674, 1037, 5902, 3735, 922, 923, 6044, 4131, 41, 1198, 187, 2877, 5952, 1987, 5956, 5445, 1095, 5447, 5962, 5451, 5965, 4558, 5841, 5843, 2901, 2011, 2012, 223, 5218, 5988, 3173, 2023, 5481, 5872, 6001, 371, 627, 1270, 2300}


In [ ]:
# Find unrated movieId
# Use set operation
# unrateddId = wholeId - ratedId
unrated_movie_id = unique_movie_id.difference(rated_movie_id)
print(unrated_movie_id)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 131098, 31, 32, 131104, 34, 30, 36, 32799, 38, 39, 40, 65577, 43, 44, 45, 46, 47, 48, 65585, 50, 49, 52, 53, 54, 55, 65588, 57, 58, 98361, 60, 61, 62, 65596, 64, 65, 66, 131130, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 163925, 92, 93, 94, 95, 65631, 97, 96, 99, 100, 101, 102, 103, 104, 105, 106, 107, 32875, 65642, 110, 111, 112, 113, 108, 65651, 116, 117, 118, 119, 121, 122, 123, 32892, 125, 126, 128, 129, 32898, 132, 135, 137, 32906, 140, 141, 163981, 144, 145, 146, 147, 65682, 148, 150, 151, 163985, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 131237, 166, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 98491, 189, 190, 191, 188, 193, 194, 195, 196, 98499, 198, 199, 98503, 201, 65738, 203, 204, 205, 65740, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 42, 

In [ ]:
# Let's create a function
def get_unrated_item(userid, rating_data):
    """
    Get unrated item id from a user id

    Parameters
    ----------
    userid : int
        The user id

    rating_data : pandas DataFrame
        The rating data

    Returns
    -------
    unrated_item_id : set
        The unrated item id
    """
    # Find the whole item id
    unique_item_id = set(rating_data['movieId'])

    # Find the item id that was rated by user id
    rated_item_id = set(rating_data.loc[rating_data['userId']==userid, 'movieId'])

    # Find the unrated item id
    unrated_item_id = unique_item_id.difference(rated_item_id)

    return unrated_item_id


In [ ]:
unrated_movie_id = get_unrated_item(userid=9, rating_data=rating_data)
print(unrated_movie_id)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 131098, 31, 32, 131104, 34, 30, 36, 32799, 38, 39, 40, 65577, 43, 44, 45, 46, 47, 48, 65585, 50, 49, 52, 53, 54, 55, 65588, 57, 58, 98361, 60, 61, 62, 65596, 64, 65, 66, 131130, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 163925, 92, 93, 94, 95, 65631, 97, 96, 99, 100, 101, 102, 103, 104, 105, 106, 107, 32875, 65642, 110, 111, 112, 113, 108, 65651, 116, 117, 118, 119, 121, 122, 123, 32892, 125, 126, 128, 129, 32898, 132, 135, 137, 32906, 140, 141, 163981, 144, 145, 146, 147, 65682, 148, 150, 151, 163985, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 131237, 166, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 98491, 189, 190, 191, 188, 193, 194, 195, 196, 98499, 198, 199, 98503, 201, 65738, 203, 204, 205, 65740, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 42, 

**Then, we create the prediction from the unrated movie**

In [ ]:
# Initialize dict
predicted_unrated_movie = {
    'userId': 9,
    'movieId': [],
    'predicted_rating': []
}

predicted_unrated_movie

{'userId': 9, 'movieId': [], 'predicted_rating': []}

In [ ]:
# Loop for over all unrated movie Id
for id in unrated_movie_id:
    # Create a prediction
    pred_id = model_best.predict(uid = predicted_unrated_movie['userId'],
                                 iid = id)

    # Append
    predicted_unrated_movie['movieId'].append(id)
    predicted_unrated_movie['predicted_rating'].append(pred_id.est)

In [ ]:
# Convert to dataframe
predicted_unrated_movie = pd.DataFrame(predicted_unrated_movie)
predicted_unrated_movie

,userId,movieId,predicted_rating
0,9,1,3.440970
1,9,2,3.178351
2,9,3,2.773542
3,9,4,2.717316
4,9,5,2.850927
...,...,...,...
9673,9,163809,3.129475
9674,9,98279,3.251486
9675,9,32743,3.143775
9676,9,65514,3.274698


Nice! Let's sort the values

In [ ]:
# Sort the predicted rating values
predicted_unrated_movie = predicted_unrated_movie.sort_values('predicted_rating',
                                                              ascending = False)

predicted_unrated_movie

,userId,movieId,predicted_rating
244,9,260,4.245551
300,9,318,4.233101
947,9,1104,4.205823
1025,9,1213,4.173759
2910,9,3451,4.144698
...,...,...,...
1967,9,2338,1.907615
3023,9,3593,1.898327
412,9,435,1.887569
4366,9,5323,1.868435


In [ ]:
# Let's create this into a function
def get_pred_unrated_item(userid, estimator, unrated_item_id):
    """
    Get the predicted unrated item id from user id

    Parameters
    ----------
    userid : int
        The user id

    estimator : Surprise object
        The estimator

    unrated_item_id : set
        The unrated item id

    Returns
    -------
    pred_data : pandas Dataframe
        The predicted rating of unrated item of user id
    """
    # Initialize dict
    pred_dict = {
        'userId': userid,
        'movieId': [],
        'predicted_rating': []
    }

    # Loop for over all unrated movie Id
    for id in unrated_item_id:
        # Create a prediction
        pred_id = estimator.predict(uid = pred_dict['userId'],
                                    iid = id)

        # Append
        pred_dict['movieId'].append(id)
        pred_dict['predicted_rating'].append(pred_id.est)

    # Create a dataframe
    pred_data = pd.DataFrame(pred_dict).sort_values('predicted_rating',
                                                     ascending = False)

    return pred_data

In [ ]:
predicted_unrated_movie = get_pred_unrated_item(userid = 9,
                                                estimator = model_best,
                                                unrated_item_id = unrated_movie_id)

predicted_unrated_movie

,userId,movieId,predicted_rating
244,9,260,4.245551
300,9,318,4.233101
947,9,1104,4.205823
1025,9,1213,4.173759
2910,9,3451,4.144698
...,...,...,...
1967,9,2338,1.907615
3023,9,3593,1.898327
412,9,435,1.887569
4366,9,5323,1.868435


And then we create the top movie predictions

In [ ]:
# Pick top k biggest rating
k = 5
top_movies_svd = predicted_unrated_movie.head(k).copy()
top_movies_svd

,userId,movieId,predicted_rating
244,9,260,4.245551
300,9,318,4.233101
947,9,1104,4.205823
1025,9,1213,4.173759
2910,9,3451,4.144698


Finally, we can add the Movie Title

In [ ]:
# Add the movie title
top_movies_svd['title'] = movie_data.loc[top_movies['movieId'], 'title'].values
top_movies_svd['genres'] = movie_data.loc[top_movies['movieId'], 'genres'].values

top_movies_svd

,userId,movieId,predicted_rating,title,genres
244,9,260,4.245551,Forrest Gump (1994),Comedy|Drama|Romance|War
300,9,318,4.233101,"Shawshank Redemption, The (1994)",Crime|Drama
947,9,1104,4.205823,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1025,9,1213,4.173759,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
2910,9,3451,4.144698,"Matrix, The (1999)",Action|Sci-Fi|Thriller


Great!
Lets wrap all to a function

In [ ]:
def get_top_highest_unrated(estimator, k, userid, rating_data, movie_data):
    """
    Get top k highest of unrated movie from a Surprise estimator RecSys

    Parameters
    ----------
    estimator : Surprise model
        The RecSys model

    k : int
        The number of Recommendations

    userid : int
        The user Id to recommend

    rating_data : pandas Data Frame
        The rating data

    movie_data : pandas DataFrame
        The movie meta data

    Returns
    -------
    top_item_pred : pandas DataFrame
        The top items recommendations
    """
    # 1. Get the unrated item id of a user id
    unrated_item_id = get_unrated_item(userid=userid, rating_data=rating_data)

    # 2. Create prediction from estimator to all unrated item id
    predicted_unrated_item = get_pred_unrated_item(userid = userid,
                                                   estimator = estimator,
                                                   unrated_item_id = unrated_item_id)

    # 3. Sort & add meta data
    top_item_pred = predicted_unrated_item.head(k).copy()
    top_item_pred['title'] = movie_data.loc[top_item_pred['movieId'], 'title'].values
    top_item_pred['genres'] = movie_data.loc[top_item_pred['movieId'], 'genres'].values

    return top_item_pred


In [ ]:
# Generate 10 recommendation for user 90
get_top_highest_unrated(estimator=model_best,
                        k=10,
                        userid=90,
                        rating_data=rating_data,
                        movie_data=movie_data)

,userId,movieId,predicted_rating,title,genres
982,90,1204,4.838250,Lawrence of Arabia (1962),Adventure|Drama|War
42,90,50,4.792610,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
760,90,912,4.750364,Casablanca (1942),Drama|Romance
2234,90,2692,4.741814,Run Lola Run (Lola rennt) (1998),Action|Crime
1425,90,1732,4.738490,"Big Lebowski, The (1998)",Comedy|Crime
2861,90,3435,4.725263,Double Indemnity (1944),Crime|Drama|Film-Noir
3080,90,3703,4.713544,"Road Warrior, The (Mad Max 2) (1981)",Action|Adventure|Sci-Fi|Thriller
639,90,741,4.704984,Ghost in the Shell (Kôkaku kidôtai) (1995),Animation|Sci-Fi
986,90,1208,4.701634,Apocalypse Now (1979),Action|Drama|War
2742,90,3275,4.699381,"Boondock Saints, The (2000)",Action|Crime|Drama|Thriller


user Id 90 is a action / thriller guy

In [ ]:
# Generate 10 recommendation for user 1000
get_top_highest_unrated(estimator=model_best,
                        k=10,
                        userid=1000,
                        rating_data=rating_data,
                        movie_data=movie_data)

,userId,movieId,predicted_rating,title,genres
1026,1000,1204,4.395384,Lawrence of Arabia (1962),Adventure|Drama|War
303,1000,318,4.369144,"Shawshank Redemption, The (1994)",Crime|Drama
785,1000,904,4.355184,Rear Window (1954),Mystery|Thriller
49,1000,50,4.313741,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
670,1000,750,4.310996,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War
998,1000,1172,4.292398,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
2526,1000,2959,4.281567,Fight Club (1999),Action|Crime|Drama|Thriller
1030,1000,1208,4.269233,Apocalypse Now (1979),Action|Drama|War
748,1000,858,4.266016,"Godfather, The (1972)",Crime|Drama
2796,1000,3275,4.260573,"Boondock Saints, The (2000)",Action|Crime|Drama|Thriller


user Id 1000 is a drama guy